In [1]:
from mxnet import gluon, init, nd, gpu, cpu, autograd
from mxnet.gluon import nn

In [2]:
class SimpleNet(nn.HybridBlock):
    def __init__(self, no4x1pooling=False, **kwargs):
        super().__init__(**kwargs)
        self.no4x1pooling = no4x1pooling
        layer_size = [min(32 * 2 ** (i + 1), 512) for i in range(6)]

        def convRelu(layer, i, bn=True):
            layer.add(nn.Conv2D(
                channels=layer_size[i], kernel_size=(3, 3), padding=(1, 1)))
            if bn:
                layer.add(nn.BatchNorm())
            layer.add(nn.LeakyReLU(alpha=0.25))
            return layer
        with self.name_scope():
            self.conv = nn.HybridSequential(prefix='')
            with self.conv.name_scope():
                self.conv = convRelu(self.conv, 0)  # bz x 64 x 32 x 280

                self.max_pool = nn.HybridSequential(prefix='')
                # bz x 128 x 16 x 140
                self.max_pool.add(nn.MaxPool2D(
                    pool_size=(2, 2), strides=(2, 2)))
                self.avg_pool = nn.HybridSequential(prefix='')
                # bz x 128 x 16 x 140
                self.avg_pool.add(nn.AvgPool2D(
                    pool_size=(2, 2), strides=(2, 2)))

                self.net = nn.HybridSequential(prefix='')
                self.net = convRelu(self.net, 1)
                # bz x 256 x 8 x 70
                self.net.add(nn.MaxPool2D(pool_size=(2, 2), strides=(2, 2)))
                self.net = convRelu(self.net, 2, True)
                self.net = convRelu(self.net, 3)
                # bz x 512 x 4 x 35
                self.net.add(nn.MaxPool2D(pool_size=(2, 2), strides=(2, 2)))
                self.net = convRelu(self.net, 4, True)
                self.net = convRelu(self.net, 5)
                self.c = 512
                if not self.no4x1pooling:
                    self.cols_pool = nn.HybridSequential(prefix='')
                    # bz x 512 x 1 x 35
                    self.cols_pool.add(nn.AvgPool2D(pool_size=(4, 1)))
                    self.cols_pool.add(nn.Dropout(rate=0.5))
                else:
                    self.c = self.c * 4
                    self.no_cols_pool = nn.HybridSequential(prefix='')
                    self.no_cols_pool.add(nn.Dropout(rate=0.5))

    def hybrid_forward(self, F, x):
        x = self.conv(x)
        max = self.max_pool(x)
        avg = self.avg_pool(x)
        x = max - avg
        x = self.net(x)
        if not self.no4x1pooling:
            x = self.cols_pool(x)
        else:
            x = self.no_cols_pool(x)
            x = F.reshape(data=x, shape=(0, -3, 1, -2))
        return x

In [9]:
net = SimpleNet()

In [10]:
x = nd.random.normal(shape=(1, 3, 32, 32))
net.initialize()

In [11]:
net.summary(x)

--------------------------------------------------------------------------------
        Layer (type)                                Output Shape         Param #
               Input                              (1, 3, 32, 32)               0
            Conv2D-1                             (1, 64, 32, 32)            1792
         BatchNorm-2                             (1, 64, 32, 32)             256
         LeakyReLU-3                             (1, 64, 32, 32)               0
         MaxPool2D-4                             (1, 64, 16, 16)               0
         AvgPool2D-5                             (1, 64, 16, 16)               0
            Conv2D-6                            (1, 128, 16, 16)           73856
         BatchNorm-7                            (1, 128, 16, 16)             512
         LeakyReLU-8                            (1, 128, 16, 16)               0
         MaxPool2D-9                              (1, 128, 8, 8)               0
           Conv2D-10        